In [ ]:
import numpy as np
import librosa as lb
import matplotlib.pyplot as plt
from math import ceil
from scipy.io import loadmat

In [ ]:
ref = loadmat('tfr_light.mat')
def diff(x, y):
    return np.linalg.norm(x.astype(y.dtype) - y)

In [ ]:
sr = 22050
hop_length = 128
win_lengths = [512, 1024, 2048]
n_fft = 2048
window = lambda n: np.hanning(n+2)[1:-1]
center = False

In [ ]:
signal_name = './Dataset/audio_exp_synth/AClassicEducation_NightOwl_STEM_08.RESYN.wav'
x , sr_orig = lb.load(signal_name, sr=None)
file_duration = len(x)/sr_orig
print(sr_orig, file_duration)

In [ ]:
diff(x, ref['x'].squeeze())

In [ ]:
from noname import spectrogram_tensor_prep

In [ ]:
%%time
spectrograms_tensor = spectrogram_tensor_prep(x, sr=sr, win_lengths=win_lengths,n_fft=n_fft, hop_length=hop_length, window=window, center=False)

In [ ]:
diff(spectrograms_tensor, ref['spectrograms_tensor'])

In [ ]:
del(spectrograms_tensor)

In [ ]:
from noname import swgm_comb

In [ ]:
%%time
#SWGM
TFR_SWGM = swgm_comb(ref['spectrograms_tensor'], 0.5)

In [ ]:
diff(TFR_SWGM, ref['TFR_SWGM'])

In [ ]:
del(TFR_SWGM)

In [ ]:
# SLS
zeta = 70 # controls the sparsity weighting process (zeta >= 200 for max(LocalSparsity))
eta = 1 # controls the energy weighting process

time_span = 0.03 # time span of the 2D window in seconds
n_cols = ceil(sr*time_span/hop_length)
size_w_S = [21, n_cols]
size_w_E = [9,  2*n_cols]



In [ ]:
from noname import spectrogram_local_sparsity

In [ ]:
%%time
TFR_SLS = spectrogram_local_sparsity(ref['spectrograms_tensor'], size_w_S, size_w_E, zeta, eta)

In [ ]:
diff(TFR_SLS, ref['TFR_SLS'])

In [ ]:
del(TFR_SLS)

In [ ]:
#LT
eta = 8 # controls the energy smearing weighting process

In [ ]:
TFR_LT = spectrogram_comb_Lukin_Todd(spectrograms_tensor, size_w_S, eta);

In [ ]:
#FHLS
eta = 20 # controls the energy weighting process
n_rows = ceil(3*n_fft/win_lengths[0])
size_W_m_k = [n_rows, n_cols]

In [ ]:
from noname import spectrogram_comb_FastHoyerLocalSparsity

In [ ]:
%%time
TFR_FHLS = spectrogram_comb_FastHoyerLocalSparsity(ref['spectrograms_tensor'], size_W_m_k, eta)

In [ ]:
diff(TFR_FHLS, ref['TFR_FHLS'])

In [ ]:
del(TFR_FHLS)

In [ ]:
#MRFCI
# Number of alphas (I, in the paper)
n_alphas = 7 # Number of alphas used for STFChTs (only alphas >= 0)

# Synthesis TFR - - - - - - - - - - - - - - - - - - -
asym_flags = [0, 0, 0] # Indicate which Nf will be used to compute Fan-chirp instances
FChT_flags = [0, 1, 1] # Indicate which Nf will be used to compute Fan-chirp instances

# Structure tensor - - - - - - - - -
R = 60 # range in dB for the ST

# G parameters
sigma_t = win_lengths[-1]/(3*sr) # in s
sigma_f = 100 # in Hz

# C_limits defines the points of reference for the linear combination procedure

# C_limits = 0 : 1/(length(Nf)-1) : 1; % Points of reference for the linear combination procedure
C_limits = np.array([[0, .25, .75, 1,], [1, 2, 2, 3]])

Nf_structure_tensor = win_lengths[0] # To compute the structure tensor

In [ ]:
from noname import mrfci

In [ ]:
%%time
TFR_MRFCI, STFT_tensor = mrfci(x, sr, win_lengths,FChT_flags, asym_flags, Nf_structure_tensor, hop_length, n_alphas, C_limits, R,sigma_t, sigma_f, center=False, interpolate=False, window=window, pad_mode='reflect')

In [ ]:
diff(TFR_MRFCI, ref['TFR_MRFCI']), diff(STFT_tensor, ref['spectrograms_tensor'])

In [ ]:
# MRFCI 2

C_limits = np.array([[0, .2, .4, 1], [1, 2, 2, 3]])

In [ ]:
TFR_MRFCI, STFT_tensor = mrfci(x, sr, win_lengths,FChT_flags, asym_flags, Nf_structure_tensor, hop_length, n_alphas, C_limits, R,sigma_t, sigma_f, center=False, interpolate=False, window=window, pad_mode='reflect'):